<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [3]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [4]:
# Posibles bandas
os.listdir("./songs_dataset/")

['prince.txt',
 'dickinson.txt',
 'notorious-big.txt',
 'beatles.txt',
 'bob-dylan.txt',
 'bjork.txt',
 'johnny-cash.txt',
 'disney.txt',
 'janisjoplin.txt',
 'kanye.txt',
 'bob-marley.txt',
 'leonard-cohen.txt',
 'ludacris.txt',
 'adele.txt',
 'alicia-keys.txt',
 'joni-mitchell.txt',
 'amy-winehouse.txt',
 'lorde.txt',
 'rihanna.txt',
 'Kanye_West.txt',
 'nirvana.txt',
 'cake.txt',
 'bieber.txt',
 'notorious_big.txt',
 'missy-elliott.txt',
 'dolly-parton.txt',
 'jimi-hendrix.txt',
 'michael-jackson.txt',
 'al-green.txt',
 'lil-wayne.txt',
 'lady-gaga.txt',
 'lin-manuel-miranda.txt',
 'nursery_rhymes.txt',
 'dj-khaled.txt',
 'radiohead.txt',
 'patti-smith.txt',
 'blink-182.txt',
 'Lil_Wayne.txt',
 'dr-seuss.txt',
 'r-kelly.txt',
 'drake.txt',
 'britney-spears.txt',
 'bruce-springsteen.txt',
 'nicki-minaj.txt',
 'kanye-west.txt',
 'paul-simon.txt',
 'nickelback.txt',
 'eminem.txt',
 'bruno-mars.txt']

In [5]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/radiohead.txt', sep='/n', header=None)
df.head()

/var/folders/ml/gmqmyfhd39bc7mggbpwznw0r0000gn/T/ipykernel_92350/2728444896.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/radiohead.txt', sep='/n', header=None)


,0
0,"Come on, come on"
1,You think you drive me crazy
2,"Come on, come on"
3,You and whose army?
4,You and your cronies


In [6]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2343


### 1 - Preprocesamiento

In [7]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [8]:
# Demos un vistazo
sentence_tokens[:2]

[['come', 'on', 'come', 'on'], ['you', 'think', 'you', 'drive', 'me', 'crazy']]

### 2 - Crear los vectores (word2vec)

In [9]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [10]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [11]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [12]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2343


In [13]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 383


### 3 - Entrenar embeddings

In [14]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=200,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 89124.28125
Loss after epoch 1: 40447.0234375
Loss after epoch 2: 38616.4609375
Loss after epoch 3: 37738.953125
Loss after epoch 4: 37809.390625
Loss after epoch 5: 37434.765625
Loss after epoch 6: 36855.21875
Loss after epoch 7: 36691.40625
Loss after epoch 8: 35232.21875
Loss after epoch 9: 35044.4375
Loss after epoch 10: 34932.375
Loss after epoch 11: 33556.0
Loss after epoch 12: 32181.15625
Loss after epoch 13: 31678.375
Loss after epoch 14: 30294.875
Loss after epoch 15: 29945.8125
Loss after epoch 16: 28267.6875
Loss after epoch 17: 27461.4375
Loss after epoch 18: 27017.875
Loss after epoch 19: 25738.25
Loss after epoch 20: 25231.4375
Loss after epoch 21: 24793.625
Loss after epoch 22: 24439.1875
Loss after epoch 23: 23323.5
Loss after epoch 24: 23092.75
Loss after epoch 25: 22049.625
Loss after epoch 26: 22376.1875
Loss after epoch 27: 21885.875
Loss after epoch 28: 21206.6875
Loss after epoch 29: 21741.5
Loss after epoch 30: 20223.125
Loss after epoch 31: 1

(1170623, 2360600)

### 4 - Ensayar

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('choke', 0.4765268862247467),
 ("payin'", 0.4705463945865631),
 ('needed', 0.4636782109737396),
 ('hearts', 0.44341522455215454),
 ('alright', 0.42987796664237976),
 ('ringing', 0.42928045988082886),
 ('its', 0.4290997087955475),
 ('rain', 0.42642396688461304),
 ("i'd", 0.4235934019088745),
 ('broken', 0.4232752025127411)]

In [16]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('uptight', 0.038635268807411194),
 ('away', -0.006682051811367273),
 ('day', -0.03571729734539986),
 ('over', -0.050668999552726746),
 ('off', -0.05396568030118942),
 ('big', -0.05529693886637688),
 ('and', -0.06878501176834106),
 ('this', -0.07278633117675781),
 ('about', -0.0790463536977768),
 ("you've", -0.0796360969543457)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["creep"], topn=10)

[('weirdo', 0.9541454911231995),
 ('ready', 0.7748377919197083),
 ('bunker', 0.6798977851867676),
 ("who's", 0.6266152262687683),
 ('such', 0.6206404566764832),
 ('stuffed', 0.6200128793716431),
 ('lie', 0.6066186428070068),
 ('height', 0.5944558382034302),
 ('great', 0.5845938324928284),
 ('alright', 0.5750374794006348)]

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["need"], topn=5)

[('needed', 0.604769229888916),
 ('attention', 0.5510879158973694),
 ('myself', 0.5088874101638794),
 ('believe', 0.5076034665107727),
 ("payin'", 0.49516627192497253)]

In [19]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["police"])

KeyError: "Key 'police' not present in vocabulary"

In [20]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("dream")
print(vector_love)

[-0.13601704  0.20232981  0.00834987 -0.26749125  0.30056277 -0.5020355
  0.327451    1.1208166   0.187635    0.00942725  0.25671086 -0.5456774
 -0.0719433  -0.20585757  0.12615837 -0.0548914  -0.18485625 -0.16165109
 -0.16811296 -0.5562529   0.10862747 -0.09662451  0.4335825   0.3574674
  0.06571554  0.48860696 -0.08798373 -0.03489013 -0.2807634   0.15898919
  0.267329   -0.47787878 -0.13828525 -0.28826666 -0.36534128 -0.05455005
  0.13331263 -0.25373155 -0.33356822 -0.14566366  0.40675855  0.40367696
  0.02460105 -0.32948408  0.34494206  0.5035701   0.56450677  0.23035869
  0.36720663  0.41672128  0.20410623 -0.22758538 -0.26233205 -0.14001192
  0.33463684  0.0475526  -0.3333087  -0.27484497  0.23262167  0.02723552
 -0.465579   -0.30705562  0.11581385  0.27590692  0.25327793  0.26715353
 -0.01743512 -0.26914006 -0.2844288  -0.30408022  0.5796684   0.0763476
 -0.17681159 -0.00811843  0.15160923  0.1767514  -0.3101527   0.00141646
 -0.02295237  0.9104636  -0.1359857   0.00794682 -0.205

In [21]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('dream', 1.0),
 ('nice', 0.6387646198272705),
 ('drop', 0.5974333882331848),
 ('bed', 0.5675229430198669),
 ('choke', 0.5611574053764343),
 ('long', 0.5409419536590576),
 ('enough', 0.5312414169311523),
 ('great', 0.509792149066925),
 ('height', 0.5045536160469055),
 ('belong', 0.5024136304855347)]

In [22]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('choke', 0.4765268862247467),
 ("payin'", 0.4705463945865631),
 ('needed', 0.4636782109737396),
 ('hearts', 0.44341522455215454),
 ('alright', 0.42987796664237976),
 ('ringing', 0.42928045988082886),
 ('its', 0.4290997087955475),
 ('rain', 0.42642396688461304),
 ("i'd", 0.4235934019088745),
 ('broken', 0.4232752025127411)]

### 5 - Visualizar agrupación de vectores

In [23]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [24]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

In [27]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.